<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/802%20code/bart-base-paradetox-split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece] datasets
!pip install transformers[torch]
!pip install sacrebleu sentencepiece
!pip install evaluate
!pip install huggingface_hub

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [28]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
from transformers import BartForConditionalGeneration

raw_datasets = load_dataset("HamdanXI/paradetox-split", split="train")
checkpoint = "facebook/bart-base"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = BartForConditionalGeneration.from_pretrained(checkpoint)

raw_datasets = raw_datasets.rename_column("en_toxic_comment", "input")
raw_datasets = raw_datasets.rename_column("en_neutral_comment", "labels")

In [29]:
raw_datasets

Dataset({
    features: ['input', 'labels'],
    num_rows: 19073
})

In [31]:
max_token_length_input = max(len(tokenizer.encode(item)) for item in raw_datasets["input"])
max_token_length_label = max(len(tokenizer.encode(item)) for item in raw_datasets["labels"])

print(f"Max token length for input: {max_token_length_input}")
print(f"Max token length for label: {max_token_length_label}")

if max_token_length_input > max_token_length_label:
    highest_length = max_token_length_input
else:
    highest_length = max_token_length_label

print(highest_length)

Max token length for input: 38
Max token length for label: 37
38


In [32]:
def tokenize_function(example):
    source = example["input"]
    target = example["labels"]

    # Tokenizing source and target without returning tensors and without padding
    tokenized_source = tokenizer(source, truncation=True, max_length=highest_length, return_tensors="pt")
    tokenized_target = tokenizer(target, truncation=True, max_length=highest_length, return_tensors="pt")

    return {
        "input_ids": tokenized_source["input_ids"][0],
        "attention_mask": tokenized_source["attention_mask"][0],
        "labels": tokenized_target["input_ids"][0]
    }

tokenized_datasets = raw_datasets.map(tokenize_function)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [33]:
tokenized_datasets

Dataset({
    features: ['input', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 19073
})

In [34]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2)

In [35]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 15258
    })
    test: Dataset({
        features: ['input', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 3815
    })
})

In [36]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(output_dir="bart-base-paradetox-split", push_to_hub=True)
training_args = training_args.set_save(steps=10000)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [37]:
trainer.train()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.253400
1000,1.099300
1500,1.077300
2000,1.005900
2500,0.820800
3000,0.842100
3500,0.830300
4000,0.767300
4500,0.677900
5000,0.676600


TrainOutput(global_step=5724, training_loss=0.8749382957389187, metrics={'train_runtime': 355.2326, 'train_samples_per_second': 128.856, 'train_steps_per_second': 16.113, 'total_flos': 599117278556160.0, 'train_loss': 0.8749382957389187, 'epoch': 3.0})

In [38]:
trainer.push_to_hub()

events.out.tfevents.1700607640.f9fee4fb60ea.2668.0:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

'https://huggingface.co/HamdanXI/bart-base-paradetox-split/tree/main/'